In [2]:
%cd ..

/home/abaric/phd/retriever-diachronic-emb


In [3]:
import os, xml, json, spacy
import xml.etree.ElementTree as ET
import pandas as pd
from tqdm import tqdm

In [26]:
nlp = spacy.load("hr_core_news_lg")

In [34]:
stone = pd.read_csv("data/stone/stone_3k.csv")[['document_id', 'text', 'aggregated_tone']]

tone_dict = {"Ton - NEG": 0,
             "Ton - NEUTR": 1,
             "Ton - POZ": 2}

stone['label'] = stone['aggregated_tone'].map(tone_dict)
stone = stone.drop(columns=['aggregated_tone'])
stone

,document_id,text,label
0,65,"Bandić favorit, a u drugi krug bi mogao ići i ...",1
1,126,"Emir se šokira kad sazna za Ferihinu laž, Gund...",0
2,180,Boo dobio konkurenciju! Da li je ovo najslađi ...,2
3,187,Ronaldo: Kladio bih se da će Leo Messi osvojit...,2
4,308,"Brill: Mobiteli su stalno uz nas, tako bi i va...",1
...,...,...,...
2875,139071,"Širimo Muzej krapinskih neandertalaca, a u dvo...",2
2876,139162,Tomislav Tolušić izašao iz istražnog zatvora,1
2877,139202,"Pale cijene nafte, trgovci strahuju od širenja...",0
2878,139350,Šef Odjela za sudsku medicinu KBC-a Split: 'Šo...,0


In [36]:
stone['label'].value_counts()

label
0    1275
2     934
1     671
Name: count, dtype: int64

In [21]:
import re

class Lemmatizer():
    def __init__(self):
        self.lemmas = {}
        # __init__ should ensure the file is downloaded on import
        self.pattern = re.compile("\w+")
        with open("data/molex/molex14_word2lemma.txt") as fin:
            for line in fin:
                word, lemma = line.split()
                self.lemmas[word] = lemma

    def lemmatize_word(self, word):
        lemma = self.lemmas.get(word.lower())
        if lemma:
            return lemma
        else:
            return word[:5]

    def lemmatize_string(self, string):
        lemmas = []
        for token in re.findall(self.pattern, string.lower()):
            lemmas.append(self.lemmatize_word(token))
        return " ".join(lemmas)
    

<>:7: SyntaxWarning: invalid escape sequence '\w'
<>:7: SyntaxWarning: invalid escape sequence '\w'
/tmp/ipykernel_187502/943492292.py:7: SyntaxWarning: invalid escape sequence '\w'
  self.pattern = re.compile("\w+")


In [19]:
def parse_word(tagged_word):
    word = tagged_word.find('Word').text
    tag = tagged_word.find('POSTag').text
    lemma = tagged_word.find('Lemma').text
    stem = tagged_word.find('BasicStem').text

    return word, tag, lemma, stem

In [37]:
molex = Lemmatizer()

In [38]:
import string

all_tags = set()
raw_documents = {}

pos_tag_words = {}

for idx, row in tqdm(stone.iterrows(), total=len(stone)):

    doc = nlp(row['text'])
    doc_tokens = []
    
    for token in doc:
        word = token.text
        pos = token.pos_.lower()
        lemma = molex.lemmatize_word(word.lower()) 
        # lemmatize
        doc_tokens.append((lemma, pos))
    
    doc_tokens = [(word, pos) for word, pos in doc_tokens if word not in string.punctuation]
        
    raw_documents[idx] = {
        'text':doc_tokens,
        'label': row['label'],
        'docid': idx,
    }



100%|██████████| 2880/2880 [00:11<00:00, 253.89it/s]


In [39]:
raw_documents

{0: {'text': [('bandić', 'propn'),
   ('favorit', 'noun'),
   ('a', 'cconj'),
   ('u', 'adp'),
   ('drugi', 'adj'),
   ('krug', 'noun'),
   ('bi', 'aux'),
   ('moći', 'verb'),
   ('ići', 'verb'),
   ('i', 'cconj'),
   ('vladimir', 'propn'),
   ('ferde', 'propn')],
  'label': 1,
  'docid': 0},
 1: {'text': [('emir', 'propn'),
   ('se', 'pron'),
   ('šokirati', 'verb'),
   ('kad', 'adv'),
   ('saznati', 'verb'),
   ('za', 'adp'),
   ('ferih', 'noun'),
   ('laž', 'noun'),
   ('gundž', 'propn'),
   ('je', 'aux'),
   ('trudan', 'adj')],
  'label': 0,
  'docid': 1},
 2: {'text': [('boo', 'propn'),
   ('dobiti', 'verb'),
   ('konkurencija', 'noun'),
   ('dati', 'sconj'),
   ('li', 'part'),
   ('je', 'aux'),
   ('ovo', 'det'),
   ('sladak', 'adj'),
   ('pas', 'noun'),
   ('na', 'adp'),
   ('svijet', 'noun')],
  'label': 2,
  'docid': 2},
 3: {'text': [('ronaldo', 'propn'),
   ('kladiti', 'verb'),
   ('bih', 'aux'),
   ('se', 'pron'),
   ('dati', 'sconj'),
   ('će', 'aux'),
   ('leo', 'propn'),

In [40]:
out_name = 'data/stone/stone_clean_tokens.jsonl'

with open(out_name, 'w') as outfile:
    for k, instance in raw_documents.items():
        outfile.write(json.dumps(instance)+"\n")